In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
from aerb_unv_reader import UNV

In [3]:
from material_prop_reader import material_section_read

In [4]:
with open('EPR_mat.pickle', 'rb') as handle:
    EPR_mat = pickle.load(handle)
with open('GROUPS.pickle', 'rb') as handle:
    GROUPS = pickle.load(handle)
with open('ELEM_GROUPS.pickle', 'rb') as handle:
    ELEM_GROUPS = pickle.load(handle)
with open('NODE_GROUPS.pickle', 'rb') as handle:
    NODE_GROUPS = pickle.load(handle)
with open('MATERIALS.pickle', 'rb') as handle:
    MATERIALS = pickle.load(handle)
with open('ELEMENT_MATERIAL_INX.pickle', 'rb') as handle:
    ELEMENT_MATERIAL_INX =  pickle.load(handle)
with open('Unique_MATERIALS.pickle', 'rb') as handle:
    Unique_MATERIALS = pickle.load(handle)

In [5]:
EPR_geo = UNV('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/CRUAS_NPP.unv')

# Shell Elements

In [6]:
## READ ALL ELEMENTS
ELEMENTS = {}
for key,value in EPR_geo.fem['elements'][91].items():
    ELEMENTS[key] = {'type':91, 'connectivity':value}
for key,value in EPR_geo.fem['elements'][94].items():
    ELEMENTS[key] = {'type':94, 'connectivity':value}

In [7]:
SHELL_ELEMENT_LIST = []
for gid, thick in EPR_mat.section_details['COQUE'].items():
    SHELL_ELEMENT_LIST = SHELL_ELEMENT_LIST + list(ELEM_GROUPS[gid])
for gid, elist in EPR_mat.section_details['SHELL'].items():
    SHELL_ELEMENT_LIST = SHELL_ELEMENT_LIST + list(ELEM_GROUPS[gid])
SHELL_ELEMENT_LIST = sorted(set(SHELL_ELEMENT_LIST))
SHELL_MAST_LIST_DF = pd.DataFrame(data=None, index=list(SHELL_ELEMENT_LIST), columns=['THICKNESS','MATERIALS'])

In [8]:
ShellGroup = {}
for gid, thick in EPR_mat.section_details['COQUE'].items():
    ShellGroup[gid] = thick[0]
for gid, elist in EPR_mat.section_details['SHELL'].items():
    ShellGroup[gid] = thick[0]

In [9]:
UniquShellGroup = {}
for gid,thick in ShellGroup.items():
    if thick in UniquShellGroup.keys():
        UniquShellGroup[thick]['elements'] = UniquShellGroup[thick]['elements'] + list(ELEM_GROUPS[gid])
    else:
        UniquShellGroup[thick] = {'elements': list(ELEM_GROUPS[gid]), 'material':{}}
    SHELL_MAST_LIST_DF.loc[UniquShellGroup[thick]['elements'],'THICKNESS'] = thick        

In [10]:
SHELL_MAST_LIST_DF = ELEMENT_MATERIAL_INX.loc[SHELL_ELEMENT_LIST]
for key, bm in UniquShellGroup.items():
    UniquShellGroup[key]['elements'] = sorted(set(UniquShellGroup[key]['elements']))
    mat_list = ELEMENT_MATERIAL_INX.loc[UniquShellGroup[key]['elements'],'MATERIALS'].unique()
    tempDB = ELEMENT_MATERIAL_INX.loc[bm['elements']]
    for mm in mat_list:
         UniquShellGroup[key]['material'][mm] = list(tempDB.index[tempDB['MATERIALS'] == mm])

In [11]:
NN = 0
for key, bm in UniquShellGroup.items():
    for matName, elist in UniquShellGroup[key]['material'].items():
        NN = NN + len(elist)

In [12]:
NN

114370

In [13]:
with open('UniquShellGroup.pickle', 'wb') as handle:
    pickle.dump(UniquShellGroup, handle, protocol=1)

In [14]:
with open('SHELL_ELEMENT_LIST.pickle', 'wb') as handle:
    pickle.dump(SHELL_ELEMENT_LIST, handle, protocol=1)

In [15]:
len(SHELL_ELEMENT_LIST)

114370

In [17]:
SHELL_MAST_LIST_DF

,MATERIALS
13030,GMT815
13031,BD5_30
13032,GMT815
13033,GMT815
13034,GMT815
...,...
127395,BN2PE80
127396,BN2PE80
127397,BN2PE80
127398,BN2PE80
